In [1]:
!pip install wandb pathling nibabel minio requests boto3

In [2]:
from utils import configure_notebook_session
import wandb

# Setup configuration variables. 
PROJECT_NAME = 'lung-ct-scan-notebook'
NOTEBOOK_NAME = 'prepare-images.ipynb'

# Configure notebook and login to wandb
configure_notebook_session(NOTEBOOK_NAME)
wandb.login()

wandb: You can find your API key in your browser here: http://wandb:8080/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR Host must be a url in the form https://some.address.com, received http://wandb:8080


True

In [3]:
# Start a new Wandb run
run = wandb.init(project=PROJECT_NAME, job_type="data_convert")

wandb: Currently logged in as: louismullie. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [4]:
from utils import query_studies, get_study_series_by_name, \
  get_series_as_nifti, display_and_log_sample_nifti_images, save_artifact_from_file

# Find a single study by ID to convert to NIfTI
study_uids = query_studies('StudyID=93725')

if study_uids:
  study_count = len(study_uids)

  print(f"Found {study_count} studies.")

  for study_uid in study_uids:

    serie = get_study_series_by_name(study_uid, 'mediastinum')
    series_instance_uid = serie['ID']
    
    print(f"Study UID: {study_uid}, serie UID: {series_instance_uid}")
    print("Getting DICOM series as NIfTI volume via Orthanc plugin")

    nifti_data = get_series_as_nifti(series_instance_uid)
    file_path = f"{series_instance_uid}.nii"

    if nifti_data:
      file_path = f"{series_instance_uid}.nii"
      with open(file_path, "wb") as file:
        file.write(nifti_data)
      print(f"Series '{series_instance_uid}' saved as '{file_path}'")
      display_and_log_sample_nifti_images(file_path, run)
      save_artifact_from_file(file_path, PROJECT_NAME, file_path, run)
      print(f"Series '{series_instance_uid}' saved to Minio/WanDB.")
    else:
      print("Error retrieving NIfTI data.")

else:
    print("No studies found.")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


http://orthanc:4242/studies?StudyID=93725 b3J0aGFuYzpvcnRoYW5jMTIz
Error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
No studies found.


In [5]:
run.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
